## 기본적인 전처리 함수들

### Dataloader용 함수

In [36]:
!pip install lightning
# 코렙용
# from google.colab import drive
# drive.mount('/content/drive')


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


### Discriminer 훈련용 데이터셋, 시계열 예측용 데이터셋을 위한 클래스

In [37]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class Discriminative2_Dataset(Dataset):
    def __init__(self, source,target,source_y,target_y, source_label,target_label):
        self.source = source
        self.target = target
        self.len = len(source)
        self.source_y=source_y
        self.target_y=target_y
        self.source_label=source_label
        self.target_label=target_label
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        src=torch.tensor(self.source[idx],dtype=torch.float32)
        tgt=torch.tensor(self.target[idx],dtype=torch.float32)
        src_y=torch.tensor(self.source_y[idx],dtype=torch.float32)
        tgt_y=torch.tensor(self.target_y[idx],dtype=torch.float32)
        src_label=torch.tensor(self.source_label[idx],dtype=torch.float32)
        tgt_label=torch.tensor(self.target_label[idx],dtype=torch.float32)

        return src,tgt,src_y,tgt_y,src_label,tgt_label

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

## minmax scaling, 대분류, 소분류 카테고리 불러오는 함수 등등

In [38]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
# lightning
import pytorch_lightning as pl
# minmax scaler
from sklearn.preprocessing import MinMaxScaler
def large_categories_getter(source_idx,target_idx):
    a=pd.read_csv('data/train.csv')
    largeones=a['대분류'].unique()
    smallones=a['소분류'].unique()
    # find row where 대분류 is largeones[0]
    largerows=a.loc[(a['대분류'] == largeones[source_idx])] 
    targets=largerows['소분류'].unique()


    targetrows=largerows.loc[(largerows['소분류'] == targets[target_idx])]
    # sum every row
    targetrows.drop(['ID','대분류','중분류','소분류','브랜드','제품'],axis=1,inplace=True) 
    targetrows=targetrows.sum(axis=0)

    # sum every row
    largerows.drop(['ID','대분류','중분류','소분류','브랜드','제품'],axis=1,inplace=True) 
    sourcerows=largerows.sum(axis=0)
    return sourcerows,targetrows


def small_categories_getter():

    a=pd.read_csv('data/train.csv')
    largeones=a['대분류'].unique()
    smallones=a['소분류'].unique()
    # find row where 대분류 is largeones[0]
    smallrows=a.loc[(a['소분류'] == smallones[0])] 
    # sum every row
    smallrows.drop(['ID','대분류','중분류','소분류','브랜드','제품'],axis=1,inplace=True) 
    totalsmallrows=smallrows.sum(axis=0)
    return totalsmallrows

def create_sequence(df, seq_length,pred_length):
    xs = []
    ys = []
    for i in range(len(df)-seq_length-pred_length):
        x = df[i:(i+seq_length)]
        y = df[(i+seq_length):(i+seq_length+pred_length)]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


def preprocessing(xs,ys):
    ## train test split
    train_len=int(len(xs)*0.7)
    train_x=xs[:train_len]
    train_y=ys[:train_len]
    test_x=xs[train_len:]
    test_y=ys[train_len:]

    # min max scaler
    x_scaler=MinMaxScaler()
    y_scaler=MinMaxScaler()
    train_x=x_scaler.fit_transform(train_x)
    train_y=y_scaler.fit_transform(train_y)
    test_x=x_scaler.transform(test_x)
    test_y=y_scaler.transform(test_y)

    # train test split
    return train_x,train_y,test_x,test_y

## Forecasting 모델 정의 (Source 훈련용)

In [39]:
import torch.nn as nn
import pytorch_lightning as pl
import torch

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #self.latent_dim = latent_dim
        
        self.encoder = nn.LSTM(self.input_dim, self.hidden_dim, batch_first=True) #used lstm 그러나 CNN, LInear 등 다른 모형도 가능
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, x):
        x=x.unsqueeze(2)
        # x shape: (batch_size, seq_length, input_dim) , input_dim=1 단변량이니까
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(self.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).to(self.device)
        out_encoder, (hn, cn) = self.encoder(x, (h0, c0))
        # 우리가 필요한건 마지막 시점에서의 hidden state
        return out_encoder


class Forecaster(pl.LightningModule):

    def __init__(self, input_dim, hidden_dim,output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #self.latent_dim = latent_dim
        self.encoder = Encoder(input_dim, hidden_dim)
        self.forecaster=nn.Sequential(
            nn.Linear(hidden_dim,hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim,output_dim)
        )
        ## 32-> 5

    def forward(self, x):
        #x=x.unsqueeze(2) # shape: (batch_size, seq_length, input_dim) 32, 30, 1(단변량이니ㄱ까)
        out_encoder=self.encoder.forward(x)
        out=self.forecaster(out_encoder[:,-1,:]) # 우리가 필요한건 마지막 시점에서의 hidden state이니까 out_encoder[:,-1,:]을 해줌
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch # 0~30-> 31~35
        y_hat=self.forward(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)
    



## discriminator 모델 정의(target 훈련용)

In [40]:
import torch
import torch.nn as nn
# lightening
import pytorch_lightning as pl
class Discriminator2(pl.LightningModule):

    def __init__(self, source_encoder,target_encoder,hidden_dim,latent_dim,input_dim,output_dim):
        super().__init__()
        self.source_encoder = source_encoder
        self.target_encoder = target_encoder
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.output_dim=output_dim
        self.automatic_optimization = False
        self.input_dim=input_dim
        self.sig=nn.Sigmoid()

        self.discriminator = nn.Sequential(
            nn.Linear(self.latent_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, 1),
        )
        self.reconstructer=nn.Sequential(
            nn.Linear(self.latent_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.input_dim),
        )
        self.forecaster=nn.Sequential(
            nn.Linear(self.latent_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, self.output_dim),
        )

    def forward(self, s,t):
        z_source = self.source_encoder(s)
        z_source=z_source[:,-1,:]
        z_target = self.target_encoder(t)
        z_target=z_target[:,-1,:]

        z = torch.cat((z_source, z_target), dim=0)
        shuffle_idx = torch.randperm(z.size(0))
        z = z[shuffle_idx]
        return self.discriminator(z.detach())
    
    def training_step(self, batch, batch_idx):

        source,target, source_y,target_y,source_label,target_label = batch # x,y refers to sequence ang label(0 or 1)
        optimizer_rs,optimizer_rt,optimizer_d=self.optimizers() ## ligthning 내장함수

        # reconstruction source loss
        optimizer_rs.zero_grad()
        optimizer_rt.zero_grad()
        z_source=self.source_encoder(source)
        z_source=z_source[:,-1,:]
        
        
        z_target=self.target_encoder(target)
        z_target=z_target[:,-1,:]



        ## 1. get reconstruction source loss
        source_reconstruction=self.reconstructer(z_source)
        source_reconstruction_loss=nn.MSELoss()(source_reconstruction,source)

        source_forecast=self.forecaster(z_source)
        source_forecast_loss=nn.MSELoss()(source_forecast,source_y)

        ## 2. get reconstruction target loss
        target_reconstruction=self.reconstructer(z_target)
        target_reconstruction_loss=nn.MSELoss()(target_reconstruction,target)

        target_forecast=self.forecaster(z_target)
        target_forecast_loss=nn.MSELoss()(target_forecast,target_y)

        rec_loss_total=source_reconstruction_loss+source_forecast_loss+target_reconstruction_loss+target_forecast_loss
        self.manual_backward(rec_loss_total)
        optimizer_rs.step()
        optimizer_rt.step()

        ## 3. get Discriminative loss
        optimizer_d.zero_grad()
        z = self.forward(source,target)
        z=z.squeeze()
        y=torch.cat((source_label,target_label),dim=0)
        d_loss = nn.BCEWithLogitsLoss()(z, y)
        self.manual_backward(d_loss)
        optimizer_d.step()

    
    def configure_optimizers(self):
        opt_rs=torch.optim.Adam(self.source_encoder.parameters(), lr=1e-4)
        opt_rt=torch.optim.Adam(self.target_encoder.parameters(), lr=1e-4)
        opt_d=torch.optim.Adam(self.discriminator.parameters(), lr=1e-4)
        opt_t=torch.optim.Adam(self.target_encoder.parameters(), lr=1e-4)

        return [opt_rs,opt_rt,opt_d],[]


## 모델  훈련 및 testing

In [41]:
source_idx=0  # 대분류 총 5개 , 0,1,2,3,4 선택 가능. 
target_idx=4 # 각 대분류마다 할당된 소분류-> 조금씩 다름
source_data,target_data=large_categories_getter(source_idx,target_idx)
#target_data,small_data=large_categories_getter(target_idx,0)


## 2. 데이터 전처리
seq_length=100 # 얼마의 기간을 가지고 다음 기간을 예측할 것인가
pred_length=20 # 다음 기간을 얼마나 예측할 것인가 ex) 다음 5일에 대한 예측치를 한번에 제공
xs,ys=create_sequence(source_data,seq_length,pred_length)
xt,yt=create_sequence(target_data,seq_length,pred_length)

# 전처리
train_xs,train_ys,test_xs,test_ys=preprocessing(xs,ys) # train test split, min max scaler    
train_xt,train_yt,test_xt,test_yt=preprocessing(xt,yt) # train test split, min max scaler

train_s_label=np.ones(len(train_xs))
train_t_label=np.zeros(len(train_xt))

discriminative_dataset=Discriminative2_Dataset(train_xs,train_xt,train_ys,train_yt,train_s_label,train_t_label)
discriminative_loader=DataLoader(discriminative_dataset,batch_size=32,shuffle=True)

# dataset, dataloader
train_dataset_source=TimeSeriesDataset(train_xs,train_ys)
test_dataset_source=TimeSeriesDataset(test_xs,test_ys)
train_loader_source=DataLoader(train_dataset_source,batch_size=32,shuffle=True)
test_loader_sourcer=DataLoader(test_dataset_source,batch_size=32,shuffle=False)




## ADDA 1. Source 모델 훈련시키기
source_model = Forecaster(input_dim=1, hidden_dim=64,output_dim=pred_length) #LSTM Encoder
source_trainer=pl.Trainer(max_epochs=300)
source_trainer.fit(source_model,train_loader_source,test_loader_sourcer)



## ADDA 2. Discriminator 훈련, Target Encoder 훈련 (번갈아 가면서)
target_encoder=Encoder(input_dim=1, hidden_dim=64)
discriminator=Discriminator2(source_model.encoder,target_encoder, hidden_dim=64, latent_dim=64,input_dim=seq_length,output_dim=pred_length)
discriminator_trainer=pl.Trainer(max_epochs=200)
discriminator_trainer.fit(discriminator,discriminative_loader)

## ADDA 3. Evaluation
# 1) Source 모델 평가
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
source_model.eval()
source_model=source_model.to(device)

test_xt=torch.tensor(test_xt,dtype=torch.float32).to(device)
#test_xt=test_xt.unsqueeze(2)
source_output=source_model.forward(test_xt)
source_output=source_output.squeeze()
source_output=source_output.detach().cpu().numpy()
# evaluate rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(test_yt[:,], source_output[:,]))
print('Test RMSE using source only: %.3f' % rmse)


# 2) Target 모델 평가
target_encoder.eval()
target_encoder=target_encoder.to(device)
target_output=target_encoder.forward(test_xt)
tar_pred=source_model.forecaster(target_output[:,-1,:])
tar_pred=tar_pred.squeeze()
tar_pred=tar_pred.detach().cpu().numpy()
# evaluate rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(test_yt[:,], tar_pred[:,]))
print('Test RMSE using target encoder trained: %.3f' % rmse)

/tmp/ipykernel_243907/3715573595.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targetrows.drop(['ID','대분류','중분류','소분류','브랜드','제품'],axis=1,inplace=True)
/tmp/ipykernel_243907/3715573595.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  largerows.drop(['ID','대분류','중분류','소분류','브랜드','제품'],axis=1,inplace=True)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/workspaces/jjh-gpu1-temp/IDSL/NSFMR/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:69: UserWarning: You passed in a

Epoch 299: 100%|██████████| 8/8 [00:00<00:00, 311.21it/s, v_num=14, train_loss_step=0.00228, train_loss_epoch=0.00296]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 8/8 [00:00<00:00, 278.69it/s, v_num=14, train_loss_step=0.00228, train_loss_epoch=0.00296]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | source_encoder | Encoder    | 17.2 K
1 | target_encoder | Encoder    | 17.2 K
2 | sig            | Sigmoid    | 0     
3 | discriminator  | Sequential | 4.2 K 
4 | reconstructer  | Sequential | 10.7 K
5 | forecaster     | Sequential | 5.5 K 
----------------------------------------------
54.6 K    Trainable params
0         Non-trainable params
54.6 K    Total params
0.219     Total estimated model params size (MB)


Epoch 299: 100%|██████████| 8/8 [00:00<00:00, 154.29it/s, v_num=15]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 8/8 [00:00<00:00, 143.36it/s, v_num=15]
Test RMSE using source only: 0.428
Test RMSE using target encoder trained: 0.389


In [42]:
test_xt.shape
source_output.shape

(102, 20)